<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import keras
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [4]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [8]:
InitSeed = 767
tf.random.set_seed(InitSeed)
np.random.seed( InitSeed)


In [9]:
x_train = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), x_train, dtype=tf.float32).numpy()

In [21]:

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=InitSeed)

In [24]:
checkpoint_cb = keras.callbacks.ModelCheckpoint (".\my_keras.model.h5" , save_best_only=True)
earlystopping_cb = keras.callbacks.EarlyStopping (patience=10, restore_best_weights=True)

epochs = 50
learning_rate = 0.001
decay_rate = learning_rate / epochs
momentum = 0.8
for train, test in kfold.split(x_train, y_train):
  input1 = keras.layers.Flatten(input_shape=[32,32,3])
  hidden1 = keras.layers.Dense(100, activation='relu', name="Hidden1")
  output = keras.layers.Dense(10, activation='softmax', name="Output" )

  model1 = keras.models.Sequential([input1, hidden1, output])

  sgd = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

  model1.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model1.fit(x_train[train], y_train[train],epochs=100, batch_size=32, validation_data=(x_train[test], y_train[test]), verbose=2, callbacks=[earlystopping_cb, checkpoint_cb])

Epoch 1/100
1407/1407 - 85s - loss: 1.8977 - accuracy: 0.3894 - val_loss: 1.5924 - val_accuracy: 0.4388
Epoch 2/100
1407/1407 - 4s - loss: 1.4964 - accuracy: 0.4753 - val_loss: 1.5473 - val_accuracy: 0.4626
Epoch 3/100
1407/1407 - 4s - loss: 1.4120 - accuracy: 0.5025 - val_loss: 1.4711 - val_accuracy: 0.4840
Epoch 4/100
1407/1407 - 4s - loss: 1.3528 - accuracy: 0.5249 - val_loss: 1.5054 - val_accuracy: 0.4730
Epoch 5/100
1407/1407 - 4s - loss: 1.3150 - accuracy: 0.5388 - val_loss: 1.5121 - val_accuracy: 0.4898
Epoch 6/100
1407/1407 - 4s - loss: 1.2713 - accuracy: 0.5551 - val_loss: 1.5260 - val_accuracy: 0.4852
Epoch 7/100
1407/1407 - 4s - loss: 1.2260 - accuracy: 0.5713 - val_loss: 1.5382 - val_accuracy: 0.4788
Epoch 8/100
1407/1407 - 4s - loss: 1.1843 - accuracy: 0.5873 - val_loss: 1.5560 - val_accuracy: 0.4826
Epoch 9/100
1407/1407 - 4s - loss: 1.1479 - accuracy: 0.5988 - val_loss: 1.5686 - val_accuracy: 0.4866
Epoch 10/100
1407/1407 - 4s - loss: 1.1147 - accuracy: 0.6095 - val_loss

KeyboardInterrupt: ignored